# MUTRITION RECOMMENDER SYSTEM

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from typing import Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')

usuariosDF = pd.read_csv('userPreferences.csv')
usuariosDF.drop('Unnamed: 0', axis=1, inplace=True)

recetasDF = pd.read_csv('Recetas_procesadas.csv')
recetasDF = recetasDF[['title', 'calories', 'protein', 'fat', 'sodium']]

In [2]:
dfmixta = usuariosDF.merge(recetasDF, on='title')

Buscar los 50 usuarios más parecidos a ti. Crear una tuple (index, similarity_score), luego ordenamos los ususarios por el similarity_score y nos quedamos solo con lso primeros 50.

In [24]:
userinfo={"peso":[80], "altura":  [160]}
dt=pd.DataFrame(userinfo)
user_sim_score=cosine_similarity(dt, usuariosDF[['peso', 'altura']])
user_sim_score=user_sim_score[0]

users_sim_score_tuple: List[tuple] = []
i = 0
for x in user_sim_score:
    users_sim_score_tuple.append((i, x))
    i+=1

sorted_by_score = sorted(users_sim_score_tuple, key=lambda tup: tup[1], reverse=True)
top_50 = sorted_by_score[:50]
#user_sim_score[::-1].sort()

# Buscar las receteas que han recomendado esos usuarios.
En la lista top_50, tenemos una tuple(index, sim_score), ahora crearemos un dataset que vincule basandose en el index la lista top_50 con la dfmixta que es la que contiene las recetas que ha recomendado cada usuario. 

In [25]:
recomendationDF = pd.DataFrame()
for i in top_50:
    recomendationDF = recomendationDF.append(dfmixta.iloc[i[0]])
recomendationDF.head(5)

,peso,altura,title,calories,protein,fat,sodium
1326,100,194,Turkey Giblet Stock,119.0,17.0,5.0,150.0
1593,85,179,Chicken Crepes with Asparagus and Mushrooms,427.0,20.0,29.0,315.0
115,76,169,Chicken Stock,41.0,3.0,3.0,81.0
282,55,151,Roast Turkey with Herbed Bread Stuffing and Gi...,2165.0,312.0,90.0,2000.0
503,85,196,Panna Cotta with Crushed Raspberry Sauce,329.0,4.0,20.0,31.0


Codificar el texto para que encuentre similitudes

In [5]:
#Importing TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Defining a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Constructing the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(recomendationDF['title'])

#Output the shape of tfidf_matrix
tfidf_matrix = tfidf.fit_transform(recomendationDF['title'])

tfidf_matrix.shape

(50, 177)

Calcular la matriz de cosine similarity de las 50 ceteas con la intencion de recomendar la receta que se asimila más con las demas.

In [6]:
# Importing linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Computing the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

cosine_sim[2] # cosine similarity de la receta [2] comparada con el resto.

array([0.42648   , 0.2332431 , 1.        , 0.        , 0.        ,
       0.28751009, 0.        , 0.20536708, 0.        , 0.        ,
       0.180641  , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.26933486, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [7]:
cosine_sim[2][0] # receta [2] comparada con la [0]

0.42647999733984837

Recorrer la matriz y sumar todos los valores, la receta que tenga la mayor suma de cosine similarity sera la que se recomiende.

In [9]:
indexLaMejor = 0
mejorResultado = 0.0
actual = 0.0

for i in range (0, 50):
    for j in range(0, 50):
        actual+=cosine_sim[i,j]
    if actual > mejorResultado:
        mejorResultado=actual
        indexLaMejor=i
        actual=0
    else:
        actual=0.0
print(indexLaMejor)

2


Ahora recoger la receta buscando por el index en el datframe de recomendaciones (que contiene la recomendacion de los top 50 usuarios mas parecidos a ti)

In [42]:
receta_recomendar=recomendationDF[indexLaMejor:indexLaMejor+1]
recetaFinal=receta_recomendar.iterrows
recetaFinal

<bound method DataFrame.iterrows of      peso  altura           title  calories  protein  fat  sodium
115    76     169  Chicken Stock       41.0      3.0  3.0    81.0>

In [ ]:
def get_recomendation(peso, altura, user_sim_matrix):
    similar_user = list(enumerate(user_sim_matrix[3]))
    #Getting the index of the movie given its title
    idx=indices[title]
    #Getting the list of cosine similarity scores
    sim_scores=list(enumerate(cosine_similarity[idx]))
    #sorting based on sim_score
    sim_scores=sorted(sim_scores,key=lambda x:x[1],reverse=True)
    # Getting the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]
    #Getting the  movies indices 
    sim_scores=[i[0] for i in sim_scores]
     
    return movies['original_title'].iloc[sim_scores]